In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Calculator") \
    .getOrCreate()

In [50]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StructType, StringType, FloatType

# tabela de fiis do mês de Julho
fiis = spark.read.csv("files/fiis/fiis_jul2024.csv", header=True, sep=";")

fiis = (fiis.withColumn("Cotacao", F.regexp_replace(F.col("Cotacao"), ",", ".").cast("double"))
           .withColumn("FFO_Yield", F.regexp_replace(F.col("FFO_Yield"), ",", ".").cast("double"))
           .withColumn("Dividend_Yield", F.regexp_replace(F.col("Dividend_Yield"), ",", ".").cast("double"))
           .withColumn("P_PER_VP", F.regexp_replace(F.col("P_PER_VP"), ",", ".").cast("double"))
           .withColumn("Liquidez", F.regexp_replace(F.col("Liquidez"), ",", ".").cast("double"))
           .withColumn("Qtd_de_imoveis", F.regexp_replace(F.col("Qtd_de_imoveis"), ",", ".").cast("double"))
           .withColumn("Preco_do_m2", F.regexp_replace(F.col("Preco_do_m2"), ",", ".").cast("double"))
           .withColumn("Aluguel_por_m2", F.regexp_replace(F.col("Aluguel_por_m2"), ",", ".").cast("double"))
           .withColumn("Cap_Rate_PERC", F.regexp_replace(F.col("Cap_Rate_PERC"), ",", ".").cast("double"))
           .withColumn("Vacancia_Media", F.regexp_replace(F.col("Vacancia_Media"), ",", ".").cast("double"))
           )

fiis = fiis.fillna(0)
fiis = fiis.select("Papel", "Segmento", "Cotacao", "FFO_Yield", "Dividend_Yield", "P_PER_VP", "Vacancia_Media")


# Dados da carteira de FIIs
FII = {
    "PAPEL": ['VINO11', 'VGIA11', 'GARE11', 'VGHF11'],
    "VALOR_POR_COTA": [0.0600, 0.1000, 0.0870, 0.0900]
}

fii_schema = StructType([
    StructField("PAPEL", StringType(), False),
    StructField("VALOR_POR_COTA", FloatType(), False)
])

FII = spark.createDataFrame(list(zip(*FII.values())), schema=fii_schema)


In [63]:
carteira = fiis.join(FII, on="PAPEL")
valor_investimento_mensal = 150.00

carteira = carteira.withColumn("MAGIC_NUMBER",  F.round(F.expr("Cotacao / VALOR_POR_COTA"),0))        \
                   .withColumn("INVESTIMENTO", F.round(F.expr("MAGIC_NUMBER * Cotacao"), 0))   \
                   .withColumn("NUM_COTAS", F.round(F.expr(f"{valor_investimento_mensal} / Cotacao"), 0))      \
                   .withColumn("RETORNO_MENSAL", F.round(F.expr("VALOR_POR_COTA * NUM_COTAS"),3))
                   
carteira.show()

+------+-------------------+-------+---------+--------------+--------+--------------+--------------+------------+------------+---------+--------------+
| Papel|           Segmento|Cotacao|FFO_Yield|Dividend_Yield|P_PER_VP|Vacancia_Media|VALOR_POR_COTA|MAGIC_NUMBER|INVESTIMENTO|NUM_COTAS|RETORNO_MENSAL|
+------+-------------------+-------+---------+--------------+--------+--------------+--------------+------------+------------+---------+--------------+
|VINO11| Lajes Corporativas|   6.78|   0.1077|           0.1|    0.63|        0.0408|          0.06|       113.0|       766.0|     22.0|          1.32|
|VGIA11|Títulos e Val. Mob.|   7.97|   0.1708|        0.1547|    0.83|           0.0|           0.1|        80.0|       638.0|     19.0|           1.9|
|GARE11|            Híbrido|   9.03|   0.0519|        0.0467|    0.95|           0.0|         0.087|       104.0|       939.0|     17.0|         1.479|
|VGHF11|            Híbrido|   8.79|   0.0434|        0.1273|    0.96|           0.0|   

## Filtrando papeis específicos

In [10]:
def carteira_desejada(carteira):

    """Verifica se a carteira desejada existe na base de consulta"""
    for i in range(0, len(carteira)):
        fiis.filter(f"Papel == '{carteira[i]}'").show()

carteira_desejada(["VINO11", "VGHF11", "GARE11", "VGIA11"])

+------+------------------+-------+---------+--------------+--------+--------------+
| Papel|          Segmento|Cotacao|FFO_Yield|Dividend_Yield|P_PER_VP|Vacancia_Media|
+------+------------------+-------+---------+--------------+--------+--------------+
|VINO11|Lajes Corporativas|   6.78|   0.1077|           0.1|    0.63|        0.0408|
+------+------------------+-------+---------+--------------+--------+--------------+

+------+--------+-------+---------+--------------+--------+--------------+
| Papel|Segmento|Cotacao|FFO_Yield|Dividend_Yield|P_PER_VP|Vacancia_Media|
+------+--------+-------+---------+--------------+--------+--------------+
|VGHF11| Híbrido|   8.79|   0.0434|        0.1273|    0.96|           0.0|
+------+--------+-------+---------+--------------+--------+--------------+

+------+--------+-------+---------+--------------+--------+--------------+
| Papel|Segmento|Cotacao|FFO_Yield|Dividend_Yield|P_PER_VP|Vacancia_Media|
+------+--------+-------+---------+-------------